# Calculate CanOE nitrate derivate

Step 2 for uptake and remineralization parameterization. First interpolated CaNOE nitrate fields (bio---NO3_interpolation.ipynb), now calculate derivative.

In [1]:
import numpy as np
import netCDF4 as nc
import datetime
from calendar import monthrange
from joblib import Parallel

Load ANHA12 coordinate file:

In [2]:
# Mesh:
mesh     = nc.Dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mesh1.nc')
tmask    = np.array(mesh.variables['tmask'])[0,:,:,:]
Z_masked = np.ma.masked_where((tmask > 0.1), tmask) 
mdepth   = np.array(mesh.variables['nav_lev'])
mlons    = np.array(mesh.variables['nav_lon'])
mlats    = np.array(mesh.variables['nav_lat'])

### Functions:

In [3]:
def load_data(year, month):
    # Load interpolated CanOE nitrate field (files created in bio---NO3_interpolation.ipynb)
    
    folder = '/ocean/brogalla/GEOTRACES/data/bio/Mn_202110/'
    filename = f'NO3_y{year}m{month:02}.nc'
    
    data  = nc.Dataset(folder+filename)
    NO3   = np.array(data.variables['NO3'])
    NO3   = NO3*1e-3*1e-3 # mmol/m3--> mol/L
    
    return NO3

In [4]:
def deriv(year, month):
    # Calculate month-to-month change in nitrate 
    
    if (month < 4) or (month > 7): # 5 and 8
        diff = np.zeros((50,2400,1632))  # Avoid replenishment of NO3 from mixing by zeroing non-summer months
    else:
        NO3_1 = load_data(year, month)    # Current month
        NO3_2 = load_data(year, month+1)  # Next month
        nday = monthrange(year, month)[1] # Number of days in current month
        
        # Calculate difference between next month and current month, NO3_2 - NO3_1 and convert units
        diff = np.subtract(NO3_2, NO3_1)/(3600*24*nday) # mol/L/month --> mol/L/s
    
    save_NO3(f'delta_prod_y{year}m{month:02}.nc', diff[:,:,:])
    return diff

In [5]:
def save_NO3(filename, field):
    
    # Save forcing files:
    ncd = nc.Dataset(f'/ocean/brogalla/GEOTRACES/data/bio/Mn_202110/{filename}', 'w', zlib=True)
    ncd.createDimension('x',1632)
    ncd.createDimension('y',2400)
    ncd.createDimension('deptht', 50)
    
    # variables
    NO3_var = ncd.createVariable('dNO3', 'float64', ('deptht','y','x'))
    NO3_var.units = 'mol/L/s'
    NO3_var.long_name = 'Month-to-month delta Nitrate'  
    NO3_var.coordinates = 'nav_lon nav_lat deptht'
    NO3_var[:] = field[:,:,:]
    
    ncd.close()
    return

## Run:

##### Choose year:

In [7]:
year = 2005

Calculate derivative:

Final result has to be in units of [NO_3]/s

\begin{equation*}
\frac{\partial{NO_{3}(i)}}{\partial{t}} \approxeq \frac{NO_{3}(i+1) - NO_{3}(i) }{\Delta t} = 
\frac{NO_{3}(2002m2)-NO_{3}(2002m1)}{(24*3600*ndays)} 
\end{equation*}

In [8]:
def joblib_solver(deriv, year, month):
    dNO3 = deriv(year, month) 
    return dNO3

In [9]:
months=np.arange(1,13)
joblist=[]
for month in months:
    positional_args=[deriv,year,month]
    keyword_args={}
    joblist.append((joblib_solver,positional_args,keyword_args))

In [10]:
ncores=1
with Parallel(n_jobs=ncores,backend='threading') as parallel:
    results=parallel(joblist)

In [8]:
i_NO3 = np.zeros((12,50,2400,1632))
for month in range(1,13):
    m = nc.Dataset(f'/ocean/brogalla/GEOTRACES/data/bio/Mn_202110/delta_prod_y2002m{month:02}.nc')
    m1 = np.array(m.variables['dNO3'])
    
    i_NO3[month-1,:,:,:] = m1